# 手动计算

In [19]:
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy import stats

In [20]:
# 读取数据
assert_file = r"D:\code\forecast_model\notebook\20240919_test_label_threshold\10y_4_5_strict_predict_n100_label_idx0_T4x2_fp16\model_dummy\159941_asset_nets_20240924.csv"
trade_file = r"D:\code\forecast_model\notebook\20240919_test_label_threshold\10y_4_5_strict_predict_n100_label_idx0_T4x2_fp16\model_dummy\159941_trades_20240924.csv"

assert_df = pd.read_csv(assert_file)
trade_df = pd.read_csv(trade_file)

In [21]:
assert_df['timestamp'] = pd.to_datetime(assert_df['timestamp'])
assert_df['date'] = assert_df['timestamp'].dt.date
assert_df

,timestamp,asset,benchmark,date
0,2024-08-29 09:35:12,1000.000,1.0555,2024-08-29
1,2024-08-29 09:35:15,999.502,1.0555,2024-08-29
2,2024-08-29 09:35:18,999.502,1.0555,2024-08-29
3,2024-08-29 09:35:21,999.502,1.0555,2024-08-29
4,2024-08-29 09:35:24,999.502,1.0555,2024-08-29
...,...,...,...,...
32199,2024-09-06 14:39:39,259.806,1.0265,2024-09-06
32200,2024-09-06 14:39:42,259.806,1.0265,2024-09-06
32201,2024-09-06 14:39:45,259.806,1.0265,2024-09-06
32202,2024-09-06 14:39:51,259.806,1.0265,2024-09-06


In [22]:
# 按日汇总
def handle_daily_data(data):
    """
    datetime
	close
    """
    return pd.Series({
        'open': data['asset'].iloc[0],
        'open_bm': data['benchmark'].iloc[0],
        'close': data['asset'].iloc[-1],
        'close_bm': data['benchmark'].iloc[-1],
    })


assert_daily_df = assert_df.groupby('date').apply(handle_daily_data)
assert_daily_df['rets'] = assert_daily_df['close'].pct_change()
assert_daily_df['rets_bm'] = assert_daily_df['close_bm'].pct_change()
assert_daily_df['rets'].iloc[0] = assert_daily_df['close'].iloc[0] / assert_df['asset'].iloc[0] - 1
assert_daily_df['rets_bm'].iloc[0] = assert_daily_df['close_bm'].iloc[0] / assert_df['benchmark'].iloc[0] - 1

assert_daily_df

,open,open_bm,close,close_bm,rets,rets_bm
date,,,,,,
2024-08-29,1000.000,1.0555,641.157,1.0555,-0.358843,0.000000
2024-08-30,642.957,1.0585,645.894,1.0655,0.007388,0.009474
2024-09-02,651.294,1.0745,650.094,1.0725,0.006503,0.006570
2024-09-03,652.494,1.0765,650.094,1.0725,0.000000,0.000000
2024-09-04,624.294,1.0295,391.111,1.0165,-0.398378,-0.052214
2024-09-05,393.511,1.0245,392.581,1.0235,0.003759,0.006886
2024-09-06,390.481,1.0165,259.696,1.0265,-0.338491,0.002931


In [23]:
# 计算回测指标
# back_test_years,annualized_return, annualized_return_bm, annualized_volatility, sharpe_ratio, max_drawdown, calmar_ratio, sortino_ratio, information_ratio, alpha, beta, value_at_risk
days_per_year = 250# a股
risk_free_rate = 0.02

back_test_years = len(assert_daily_df) / days_per_year
annualized_return = (assert_daily_df['close'].iloc[-1] / assert_daily_df['open'].iloc[0]) ** (1 / back_test_years) - 1
annualized_return_bm = (assert_daily_df['close_bm'].iloc[-1] / assert_daily_df['open_bm'].iloc[0]) ** (1 / back_test_years) - 1
annualized_volatility= assert_daily_df['rets'].std() * np.sqrt(days_per_year)
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
cumulative_returns = (1 + assert_daily_df['rets']).cumprod()
peak = cumulative_returns.cummax()
max_drawdown = ((peak - cumulative_returns) / peak).fillna(0).max()
calmar_ratio = annualized_return / max_drawdown
sortino_ratio = (annualized_return - risk_free_rate) / (assert_daily_df['rets'][assert_daily_df['rets'] < 0].std() * np.sqrt(days_per_year))
tracking_error = (assert_daily_df['rets'] - assert_daily_df['rets_bm']).std() * np.sqrt(250)
information_ratio = (annualized_return - annualized_return_bm) / tracking_error
beta = assert_daily_df.cov().loc['rets', 'rets_bm'] / (assert_daily_df['rets_bm'].var())
alpha = annualized_return - (risk_free_rate + beta * (annualized_return_bm - risk_free_rate))
value_at_risk = -(assert_daily_df['rets'].mean() + stats.norm.ppf(0.95) * assert_daily_df['rets'].std())

annualized_return_bm, annualized_return,annualized_volatility,sharpe_ratio,max_drawdown,calmar_ratio, sortino_ratio,information_ratio,alpha,beta,value_at_risk

(-0.6302716807441413,
 -1.0,
 3.1366711806347034,
 -0.32518550439628924,
 0.6005254624715811,
 -1.665208325862325,
 -2.118481136390538,
 -0.1258426468653898,
 2.648979840477148,
 5.642226086608192,
 -0.172298022479415)

In [24]:
trade_df

,symbol,open_order_timestamp,open_deal_timestamp,open_order_price,open_order_vol,open_deal_price,open_deal_vol,open_fee,close_order_timestamp,close_deal_timestamp,close_order_price,close_order_vol,close_deal_price,close_deal_vol,close_fee,total_fee,profit,profit_t
0,159941,2024-08-29 09:35:12.000,2024-08-29 09:35:15.000,1.797690e+308,900,1.056,900,0.047520,2024-08-29 09:35:36.000,2024-08-29 09:35:39.000,0,900,1.055,900,0.047475,0.094995,-0.099500,-0.099500
1,159941,2024-08-29 09:36:30.000,2024-08-29 09:36:33.000,1.797690e+308,900,1.056,900,0.047520,2024-08-29 09:36:51.000,2024-08-29 09:36:54.000,0,900,1.055,900,0.047475,0.094995,-0.099599,-0.198999
2,159941,2024-08-29 09:37:21.000,2024-08-29 09:37:24.000,1.797690e+308,900,1.057,900,0.047565,2024-08-29 09:37:39.000,2024-08-29 09:37:42.000,0,900,1.056,900,0.047520,0.095085,-0.099707,-0.298507
3,159941,2024-08-29 09:37:45.000,2024-08-29 09:37:48.000,1.797690e+308,900,1.057,900,0.047565,2024-08-29 09:37:57.000,2024-08-29 09:38:00.000,0,900,1.056,900,0.047520,0.095085,-0.099806,-0.398016
4,159941,2024-08-29 09:38:06.000,2024-08-29 09:38:09.000,1.797690e+308,900,1.057,900,0.047565,2024-08-29 09:38:18.000,2024-08-29 09:38:21.000,0,900,1.056,900,0.047520,0.095085,-0.099906,-0.497524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,159941,2024-09-06 14:33:27.000,2024-09-06 14:33:30.000,1.797690e+308,200,1.027,200,0.010270,2024-09-06 14:34:06.000,2024-09-06 14:34:09.000,0,200,1.025,200,0.010250,0.020520,-0.161243,-73.962200
1400,159941,2024-09-06 14:34:12.000,2024-09-06 14:34:15.000,1.797690e+308,200,1.027,200,0.010270,2024-09-06 14:34:24.000,2024-09-06 14:34:27.000,0,200,1.026,200,0.010260,0.020530,-0.084696,-73.984300
1401,159941,2024-09-06 14:34:30.000,2024-09-06 14:34:33.000,1.797690e+308,200,1.026,200,0.010260,2024-09-06 14:34:36.000,2024-09-06 14:34:39.000,0,200,1.025,200,0.010250,0.020510,-0.084760,-74.006300
1402,159941,2024-09-06 14:34:42.000,2024-09-06 14:34:45.000,1.797690e+308,200,1.026,200,0.010260,2024-09-06 14:34:57.000,2024-09-06 14:35:00.000,0,200,1.025,200,0.010250,0.020510,-0.084832,-74.028400


In [25]:
# number_of_trades, profit_factor, win_rate
number_of_trades =len(trade_df) 
win_trades = trade_df.loc[trade_df['profit'] > 0, 'profit']
loss_trades = trade_df.loc[trade_df['profit'] < 0, 'profit']
profit_factor = win_trades.sum() / loss_trades.sum()
win_rate = len(win_trades) / number_of_trades

number_of_trades, profit_factor,win_rate

(1404, -0.0020018147634588163, 0.0007122507122507123)

# Quantstats 库

In [26]:
%matplotlib inline
import quantstats as qs

In [27]:
# funcs = iter([i for i in dir(qs.stats) if not i.startswith('_')])

In [28]:
# func_name = next(funcs)
# print(func_name)

# func = getattr(qs.stats, func_name)
# func?

In [29]:
ret = assert_daily_df
ret['returns'] = assert_daily_df['rets']
ret['benchmark'] = assert_daily_df['rets_bm']
ret.index = pd.to_datetime(ret.index)
ret

,open,open_bm,close,close_bm,rets,rets_bm,returns,benchmark
date,,,,,,,,
2024-08-29,1000.000,1.0555,641.157,1.0555,-0.358843,0.000000,-0.358843,0.000000
2024-08-30,642.957,1.0585,645.894,1.0655,0.007388,0.009474,0.007388,0.009474
2024-09-02,651.294,1.0745,650.094,1.0725,0.006503,0.006570,0.006503,0.006570
2024-09-03,652.494,1.0765,650.094,1.0725,0.000000,0.000000,0.000000,0.000000
2024-09-04,624.294,1.0295,391.111,1.0165,-0.398378,-0.052214,-0.398378,-0.052214
2024-09-05,393.511,1.0245,392.581,1.0235,0.003759,0.006886,0.003759,0.006886
2024-09-06,390.481,1.0165,259.696,1.0265,-0.338491,0.002931,-0.338491,0.002931


In [30]:
p = (ret['returns'] + 1).cumprod()
p


date
2024-08-29    0.641157
2024-08-30    0.645894
2024-09-02    0.650094
2024-09-03    0.650094
2024-09-04    0.391111
2024-09-05    0.392581
2024-09-06    0.259696
Name: returns, dtype: float64

In [52]:
qs.stats.greeks(ret['returns'], ret['benchmark'], periods=250)

beta      5.642226
alpha   -33.191857
dtype: float64

In [47]:
# backtrader夏普率的计算方式
# # 先把年化的无风险收益率降级到每日的无风险收益率，然后计算每日的超额收益率，计算每日超额收益率的平均值和标准差，
# # 用平均值除以标准差就计算得到日夏普率,然后转化成年夏普率
day_risk_free_rate = (1+0.02)**(1/250)-1
day_excess_rate = ret['returns']-day_risk_free_rate
avg_day_excess_rate = day_excess_rate.mean()
std_day_excess_rate = day_excess_rate.std()
day_sharpe = avg_day_excess_rate/std_day_excess_rate
year_sharpe = day_sharpe*250**0.5
print(f"按照backtrader的方法计算出来的夏普率为:{year_sharpe}")# 按照backtrader的方法计算出来的夏普率为:2.7833366546477807

按照backtrader的方法计算出来的夏普率为:-12.281179144541193


In [53]:
print('annualized_return: ', qs.stats.cagr(returns=ret['returns'], rf=0.02,periods=250))
print('annualized_return_bm: ', qs.stats.cagr(returns=ret['benchmark'], rf=0.02,periods=250))
print('annualized_volatility: ', qs.stats.volatility(returns=ret['returns'],periods=250))
print('sharpe_ratio: ', qs.stats.sharpe(returns=ret['returns'], rf=0.02,periods=250))
print('max_drawdown: ', qs.stats.max_drawdown(ret['returns']))
print('calmar_ratio: ', qs.stats.calmar(ret['returns']))
print('sortino_ratio: ', qs.stats.sortino(returns=ret['returns'], rf=0.02,periods=250))
print('information_ratio: ', qs.stats.information_ratio(ret['returns'], ret['benchmark']))
alphabeta = qs.stats.greeks(ret['returns'], ret['benchmark'], periods=250)
print('alpha',alphabeta.alpha)
print('beta',alphabeta.beta)
print('profit_factor: ', qs.stats.profit_factor(ret['returns']))
print('value_at_risk: ', qs.stats.value_at_risk(ret['returns']))

annualized_return:  -1.0
annualized_return_bm:  -0.5813053833844422
annualized_volatility:  3.1366711806347034
sharpe_ratio:  -12.281179144541193
max_drawdown:  -0.6005254624715812
calmar_ratio:  -1.6652083258623245
sortino_ratio:  -10.16376918100443
information_ratio:  -0.8085607951526866
alpha -33.19185680267409
beta 5.642226086608193
profit_factor:  0.016107657842279858
value_at_risk:  -0.48031576068172993


: 

In [33]:
annualized_return, annualized_return_bm, annualized_volatility, sharpe_ratio, max_drawdown, alpha, beta
cal_data = {
    'annualized_return': annualized_return,
    'annualized_return_bm': annualized_return_bm,
    'annualized_volatility': annualized_volatility,
    'sharpe_ratio': sharpe_ratio,
    'max_drawdown': max_drawdown,
    'calmar_ratio': calmar_ratio,
    'sortino_ratio': sortino_ratio,
    'information_ratio': information_ratio,
    'alpha': alpha,
    'beta': beta,
    'profit_factor': profit_factor,
    'value_at_risk': value_at_risk,
}

for k,v in cal_data.items():
    print(f'{k}: {v}')

annualized_return: -1.0
annualized_return_bm: -0.6302716807441413
annualized_volatility: 3.1366711806347034
sharpe_ratio: -0.32518550439628924
max_drawdown: 0.6005254624715811
calmar_ratio: -1.665208325862325
sortino_ratio: -2.118481136390538
information_ratio: -0.1258426468653898
alpha: 2.648979840477148
beta: 5.642226086608192
profit_factor: -0.0020018147634588163
value_at_risk: -0.172298022479415


In [34]:
qs.stats.value_at_risk?

Signature:
qs.stats.value_at_risk(
    returns,
    sigma=1,
    confidence=0.95,
    prepare_returns=True,
)
Docstring:
Calculats the daily value-at-risk
(variance-covariance calculation with confidence n)
File:      d:\programs\anaconda3\lib\site-packages\quantstats\stats.py
Type:      function